# Purge Un-Verified Images

### Ref:  camera-api/example_utils.py  
(which came from ssd-dag)  

1. use labelImg to verify
2. move xml to annotation dir
3. move jpg to jpeg_images dir
4. OPTIONAL - make tf records
5. this program will read annotation
  - move unverified annotation to trash/
  - move unverified jpeg to trash/
  
 

In [1]:
import os
import sys
import shutil

In [2]:
# append to path so it wil find the tensorflow utilities

cwd = os.getcwd()
# this path is different in this project
models = os.path.abspath(os.path.join(cwd, '..', 'models/research/'))
slim = os.path.abspath(os.path.join(cwd, '..', 'models/research/slim'))
sys.path.append(models)
sys.path.append(slim)


import gen_imagesets
import example_utils

In [ ]:
## Global
DATA_DATE = '20200602'

# project directories
PROJECT = os.getcwd()
HSDATA = '/hsdata'     # high speed data

IMAGE_DIR = os.path.join(HSDATA, "jpeg_images")
ANNOTATION_DIR_ROOT = os.path.join(HSDATA, "annotation")
TRASH_DIR = os.path.join(PROJECT, "trash")
TRAINING_SPLIT_TUPLE =  (100,0,0)


In [ ]:
print ("annotation root:", ANNOTATION_DIR_ROOT)
verified_list, not_verified_list = gen_imagesets.gen_verified_lists(ANNOTATION_DIR_ROOT)
print ("verified list length:", len(verified_list))
print ("non-verified list length:", len(not_verified_list))

In [ ]:
# -- make sure you hvae a trash directory
#    camera-api/trash

# move unverfied to trash
for i, image_id in enumerate(not_verified_list):
    if image_id == '.gitkeep':
        continue
        
    xml_source, jpg_source = example_utils.get_image_annotation_paths(image_id)

    shutil.move(xml_source, TRASH_DIR)
    shutil.move(jpg_source, TRASH_DIR)


In [ ]:
! ls trash | wc

# Backup to S3

In [ ]:
tarball_name = DATA_DATE + "_annotation.tar.gz"
annotation_source = os.path.join(HSDATA, 'annotation')
! tar czvf $tarball_name $annotation_source

In [ ]:
! aws s3 cp $tarball_name s3://jmduff.security-system/training_data/ --profile=jmduff

In [ ]:
tarball_name = DATA_DATE + "_jpeg_images.tar.gz"
jpeg_images_source = os.path.join(HSDATA, 'jpeg_images')
! tar cf - $jpeg_images_source | pigz > $tarball_name

In [ ]:
! aws s3 cp $tarball_name s3://jmduff.security-system/training_data/ --profile=jmduff